In [2]:
!pip install tabula-py
!pip install JPype1
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.2 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
import os
import pdfplumber
import re
import json

# Input and output directories
pdf_directory = '/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Input'
output_directory = '/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Output'

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

def extract_date(text):
    """Extracts the first occurrence of a date in 'Month Day, Year' format."""
    date_match = re.search(r'\b[A-Z][a-z]+ \d{1,2}, \d{4}\b', text)
    return date_match.group() if date_match else None

def clean_text(text):
    """Removes footnotes, references, newlines, and unnecessary spaces."""
    # Remove numbered footnotes (e.g., "1. Some text here.")
    text = re.sub(r'\n\d+\s+.*?\.', '', text)

    # Remove footnotes between a horizontal line and a page number (e.g., "- 2 -")
    text = re.sub(r'(?s)\n_{5,}.*?-\s*\d+\s*-\n', '', text)

    # Remove inline citations like [5], (2016), or superscripts
    text = re.sub(r'\[\d+\]|\(\d{4}\)|\d+\s', '', text)

    # Remove references and similar sections
    text = re.split(r'(?i)\nReferences|\nBibliography|\nWorks Cited', text)[0]

    # Remove expressions of gratitude
    text = re.sub(r'I am grateful to .*? for .*?text\.', '', text)

    # Remove excessive newlines and replace them with space
    text = re.sub(r'\n+', ' ', text)

    # Remove excessive spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove the specific phrase related to ADB internal information
    text = re.sub(r'INTERNAL\. This information is accessible to ADB Management and Staff\. It may be shared outside ADB with appropriate permission\.', '', text)

    return text

# Loop through each PDF file in the input directory
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        output_json_path = os.path.join(output_directory, os.path.splitext(pdf_file)[0] + '.json')

        with pdfplumber.open(pdf_path) as pdf:
            full_text = " ".join([page.extract_text() for page in pdf.pages if page.extract_text()])

        # Extract date from the first valid occurrence
        date = extract_date(full_text)

        # Clean the extracted content
        cleaned_content = clean_text(full_text)

        # Store extracted information
        data = {"date": date, "content": cleaned_content}

        # Save to JSON file
        with open(output_json_path, "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, indent=4, ensure_ascii=False)

        print(f"Converted '{pdf_file}' to '{output_json_path}' ✅")


Converted 'Apr 10 2007.pdf' to '/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Output/Apr 10 2007.json' ✅


In [15]:
import os
import json

def merge_json_files(folder_path, output_file):
    merged_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                merged_data.append(json_data)

    with open(output_file, 'w', encoding='utf-8') as output_file:
        json.dump(merged_data, output_file, ensure_ascii=False, indent=2)

    print(f'Merging completed. Merged data written to {output_file}')

if __name__ == "__main__":
    input_folder = r'/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Output'
    output_file = r'/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Output/Merged.json'

    merge_json_files(input_folder, output_file)


Merging completed. Merged data written to <_io.TextIOWrapper name='/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Output/Merged.json' mode='w' encoding='utf-8'>


In [16]:
import pandas as pd

# File path for the merged JSON file
json_file_path = r'/content/drive/My Drive/Final Draft Thesis/Replication Code/Monetary Policy Stance (MPS) Code/Speeches/Output/Merged.json'

# Read the JSON file into a Pandas DataFrame
df = pd.read_json(json_file_path, orient='records')

# Convert the "date" column to datetime format, errors='coerce' will replace invalid formats with NaT (Not a Time)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop rows where the date conversion failed (i.e., rows with invalid date formats)
df = df.dropna(subset=['date'])

# Display the DataFrame
df.tail(100)

,date,content
0,2016-09-12,For release on delivery 1:p.m. EDT (12:p.m. CD...
1,1997-01-05,Remarks by Governor Laurence H. Meyer At the A...


Removing Footnotes

In [ ]:
import pandas as pd
import re

# Function to remove footnotes
def remove_footnotes(text):
    # Regular expression to match footnotes that start with a space followed by a digit and a capital letter, and end with a period or dash
    text = re.sub(r'(?<!\d)\s\d+\s+[A-Z].*?(?:\.|-)', '', text)

    # Return cleaned text
    return text

# Apply the function to the 'content' column
df['content'] = df['content'].apply(remove_footnotes)

# Display the first 100 rows of the updated dataframe
df.head(100)


,date,content
0,2016-09-12,For release on delivery 1:15 p.m. EDT (12:15 p...


First Pass Cleaning

In [ ]:
import re
import pandas as pd

# Define a function to remove specific text and phrases
def remove_footnotes_references(text):
    # Remove text between "Footnotes" and "Last Update:"
    text = re.sub(r'Footnotes.*?Last Update:', 'Last Update:', text, flags=re.DOTALL | re.IGNORECASE)

    # Remove text between "References" and "Last Update:"
    text = re.sub(r'References.*?Last Update:', 'Last Update:', text, flags=re.DOTALL | re.IGNORECASE)

    # Pattern to match the specific internal note phrase
    pattern_internal_note = r'INTERNAL\. This information is accessible to ADB Management and staff\. It may be shared outside ADB with appropriate permission\.'
    text = re.sub(pattern_internal_note, '', text, flags=re.IGNORECASE | re.DOTALL)

    # Pattern to match and remove the updated ADB phrase
    pattern_adb_phrase = r'This information is accessible to ADB Management and staff\. It may be shared outside ADB with appropriate permission\.'
    text = re.sub(pattern_adb_phrase, '', text, flags=re.IGNORECASE | re.DOTALL)

    # Find the last occurrence of "Return to top" and remove text after it
    last_return_to_top = text.rfind('Return to top')
    if last_return_to_top != -1:
        text = text[:last_return_to_top + len('Return to top')]

    return text

# Apply the function to the 'content' column
df['content'] = df['content'].apply(remove_footnotes_references)

# Display the first 100 rows of the modified DataFrame
df.head(100)


,date,content
0,2016-09-12,For release on delivery 1:15 p.m. EDT (12:15 p...


Second Pass Cleaning

In [ ]:
import pandas as pd
import re



# Regex pattern to match date at the beginning of the content
date_pattern = r"^\b\w+\s\d{1,2},\s\d{4},?\s?"

# Regex pattern to match "For release" and everything until the date format Month Day, Year
for_release_pattern = r"For release.*?\b\w+\s\d{1,2},\s\d{4}"

# Remove the date pattern at the beginning of the content
df['content'] = df['content'].str.replace(date_pattern, "", regex=True)

# Remove the "For release" pattern and everything until the date
df['content'] = df['content'].str.replace(for_release_pattern, "", regex=True)

# Remove the phrase "Remarks by"
df['content'] = df['content'].str.replace("Remarks by", "", regex=False)

df.head(100)


,date,content
0,2016-09-12,For release on delivery 1:15 p.m. EDT (12:15 p...


In [ ]:
df.to_csv('/content/drive/My Drive/Consultancy ADB/Fed/Speeches/Output/raw_meyer.csv', index=False)